# Entity Metadata Extraction

이 데모에서는 새로운 EntityExtractor를 사용하여 메타데이터에 저장된 각 노드에서 엔터티를 추출합니다. 

기본 모델은 tomaarsen입니다.

In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from dotenv import load_dotenv
load_dotenv()

True

#### Load the data

In [2]:
import json

with open('E-9_Visa_Docs.json', 'r') as f:
    visa_docs_json = json.load(f)


""" Load Documents from JSON file """

from llama_index.core.schema import Document
visa_docs = [
    Document(
        text=doc["text"], 
        metadata=doc["metadata"], 
        excluded_llm_metadata_keys=doc["excluded_llm_metadata_keys"], 
        excluded_embed_metadata_keys=doc["excluded_embed_metadata_keys"],
    ) 
    for doc in visa_docs_json
]

#### Setup the Extractor and Parser

In [3]:
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.node_parser import SentenceSplitter

entity_extractor = EntityExtractor(
    prediction_threshold=0.5,
    label_entities=False,  # include the entity label in the metadata (can be erroneous)
    device="cpu",  # set to "cuda" if you have a GPU
)

node_parser = SentenceSplitter()

transformations = [node_parser, entity_extractor]

INFO:datasets:PyTorch version 2.2.1 available.
PyTorch version 2.2.1 available.


config.json:   0%|          | 0.00/5.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

#### Extracting Metadata

CPU에서 실행되지 않으므로 지금은 문서의 하위 집합에서만 실행하겠습니다. 그래도 모든 문서에 대해 자유롭게 실행해 보세요!

In [10]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
from llama_index.core.ingestion import IngestionPipeline


pipeline = IngestionPipeline(transformations=transformations)
nodes = pipeline.run(documents=visa_docs[1:3])

Extracting entities:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
for doc in visa_docs[1:3]:
    print(doc.metadata)

{'page_label': 2, 'total_pages': 19, 'document_title': '고용허가제 법률적 개요와 고용허가제 선정국가', 'questions_this_excerpt_can_answer': '1. 고용허가제에 따라 외국인 근로자가 한국에서 최장으로 취업할 수 있는 기간은 얼마인가요?\n2. 고용허가제 선정국가는 총 몇 개국이며, 그 중에는 어떤 나라들이 포함되어 있나요?', 'prev_section_summary': '이 문서는 E-9 비자 가이드에 대한 내용을 담고 있습니다. 고용허가제 해당자의 사증 발급 절차, 허용 업종 및 체류 자격, 사증 발급 인정서 발급 절차 등에 대해 설명하고 있습니다. 또한, 고용허가제 해당자가 근무처를 변경할 수 있는 조건과 그 절차, 필요한 제출 서류 등에 대한 정보도 포함하고 있습니다. 이 외에도 고용허가제 해당자의 체류자격 변경 허가, 체류 기간 연장 허가, 재입국 허가, 외국인 등록, 고용 변동 신고 등에 대한 내용도 다루고 있습니다.', 'section_summary': '이 문서는 한국의 고용허가제에 대한 법률적 개요와 고용허가제 선정국가에 대해 설명하고 있습니다. 고용허가제는 외국인 근로자에게 최장 4년 10개월간 취업을 허용하는 인력제도로, 사업주에게 외국인 근로자의 고용을 허가합니다. 이 제도는 2004년 8월부터 시행되어 왔으며, 현재까지 16개국과 MOU를 체결하고 있습니다. 또한, 상시 근로자(고용보험 기준) 300인 미만 또는 자본금 80억원 이하의 중소기업에 외국인 고용허가를 내주고 있습니다. 고용허가제 선정국가로는 태국, 필리핀, 스리랑카, 베트남, 인도네시아, 몽골, 파키스탄, 우즈베키스탄, 캄보디아, 중국, 방글라데시, 네팔, 미얀마, 키르기스스탄, 동티모르, 라오스 등 총 16개국이 포함되어 있습니다.', 'excerpt_keywords': '고용허가제, 외국인 근로자, 취업 허용 기간, 선정국가, 중소기업'}
{'page_label': 3, 'total_pages': 1

In [16]:
for node in nodes:
    print(node.metadata)

{'page_label': 2, 'total_pages': 19, 'document_title': '고용허가제 법률적 개요와 고용허가제 선정국가', 'questions_this_excerpt_can_answer': '1. 고용허가제에 따라 외국인 근로자가 한국에서 최장으로 취업할 수 있는 기간은 얼마인가요?\n2. 고용허가제 선정국가는 총 몇 개국이며, 그 중에는 어떤 나라들이 포함되어 있나요?', 'prev_section_summary': '이 문서는 E-9 비자 가이드에 대한 내용을 담고 있습니다. 고용허가제 해당자의 사증 발급 절차, 허용 업종 및 체류 자격, 사증 발급 인정서 발급 절차 등에 대해 설명하고 있습니다. 또한, 고용허가제 해당자가 근무처를 변경할 수 있는 조건과 그 절차, 필요한 제출 서류 등에 대한 정보도 포함하고 있습니다. 이 외에도 고용허가제 해당자의 체류자격 변경 허가, 체류 기간 연장 허가, 재입국 허가, 외국인 등록, 고용 변동 신고 등에 대한 내용도 다루고 있습니다.', 'section_summary': '이 문서는 한국의 고용허가제에 대한 법률적 개요와 고용허가제 선정국가에 대해 설명하고 있습니다. 고용허가제는 외국인 근로자에게 최장 4년 10개월간 취업을 허용하는 인력제도로, 사업주에게 외국인 근로자의 고용을 허가합니다. 이 제도는 2004년 8월부터 시행되어 왔으며, 현재까지 16개국과 MOU를 체결하고 있습니다. 또한, 상시 근로자(고용보험 기준) 300인 미만 또는 자본금 80억원 이하의 중소기업에 외국인 고용허가를 내주고 있습니다. 고용허가제 선정국가로는 태국, 필리핀, 스리랑카, 베트남, 인도네시아, 몽골, 파키스탄, 우즈베키스탄, 캄보디아, 중국, 방글라데시, 네팔, 미얀마, 키르기스스탄, 동티모르, 라오스 등 총 16개국이 포함되어 있습니다.', 'excerpt_keywords': '고용허가제, 외국인 근로자, 취업 허용 기간, 선정국가, 중소기업'}
{'page_label': 3, 'total_pages': 1